# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('zillow_data.csv')

In [3]:
df.sort_values(by='RegionName').head(20)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5850,58196,1001,Agawam,MA,Springfield,Hampden,5851,113100.0,112800.0,112600.0,...,213900,215700,218200,220100,221100,221700,221700,221700,222700,223600
4199,58197,1002,Amherst,MA,Springfield,Hampshire,4200,161000.0,160100.0,159300.0,...,333700,334800,336700,338900,340400,342000,344500,347400,350600,353300
11213,58200,1005,Barre,MA,Worcester,Worcester,11214,103100.0,103400.0,103600.0,...,205600,206800,208800,210400,211300,213300,215600,217900,219500,220700
6850,58201,1007,Belchertown,MA,Springfield,Hampshire,6851,133400.0,132700.0,132000.0,...,266100,266300,267000,267500,268000,268100,268100,268800,270000,270600
14547,58202,1008,Blandford,MA,Springfield,Hampden,14548,117500.0,117300.0,117100.0,...,202400,202900,205900,208500,207500,205400,204500,206800,210900,214200
11702,58204,1010,Brimfield,MA,Springfield,Hampden,11703,115800.0,115700.0,115500.0,...,246400,247500,249100,248800,245400,243800,244600,246700,248800,250900
14475,58205,1011,Chester,MA,Springfield,Hampden,14476,87000.0,87000.0,87100.0,...,160200,156700,156600,157700,155400,151400,150100,150100,149500,149100
4568,58207,1013,Chicopee,MA,Springfield,Hampden,4569,88500.0,88500.0,88400.0,...,167700,169300,171100,171900,172300,173300,174600,175900,176900,177400
2694,58209,1020,Chicopee,MA,Springfield,Hampden,2695,97200.0,97100.0,97000.0,...,178600,181000,184100,186100,186800,187400,188100,189000,189600,189800
14594,58212,1026,Cummington,MA,Springfield,Hampshire,14595,91200.0,91100.0,91000.0,...,193400,195900,200800,202600,201800,201200,204600,211200,217900,220800


# Step 2: Data Preprocessing

In [4]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [5]:
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

Note:  I have surmised that RegionName is the ZIP code for each entry.  RegionName values with only 4 digits represent ZIP codes that actually begin with '0'.  So that I can work with ZIP codes in the data set, I will need to add that zero onto every 4-digit RegionName value.  Once that's completed, I'll rename this column "ZipCode".  

In [6]:
df.RegionName = df.RegionName.astype('int64')
df.RegionName.dtype

dtype('int64')

In [7]:
# df.RegionName.sort_values().head(10)

df.sort_values(by="RegionName").head(20)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
5850,58196,1001,Agawam,MA,Springfield,Hampden,5851,113100.0,112800.0,112600.0,...,213900,215700,218200,220100,221100,221700,221700,221700,222700,223600
4199,58197,1002,Amherst,MA,Springfield,Hampshire,4200,161000.0,160100.0,159300.0,...,333700,334800,336700,338900,340400,342000,344500,347400,350600,353300
11213,58200,1005,Barre,MA,Worcester,Worcester,11214,103100.0,103400.0,103600.0,...,205600,206800,208800,210400,211300,213300,215600,217900,219500,220700
6850,58201,1007,Belchertown,MA,Springfield,Hampshire,6851,133400.0,132700.0,132000.0,...,266100,266300,267000,267500,268000,268100,268100,268800,270000,270600
14547,58202,1008,Blandford,MA,Springfield,Hampden,14548,117500.0,117300.0,117100.0,...,202400,202900,205900,208500,207500,205400,204500,206800,210900,214200
11702,58204,1010,Brimfield,MA,Springfield,Hampden,11703,115800.0,115700.0,115500.0,...,246400,247500,249100,248800,245400,243800,244600,246700,248800,250900
14475,58205,1011,Chester,MA,Springfield,Hampden,14476,87000.0,87000.0,87100.0,...,160200,156700,156600,157700,155400,151400,150100,150100,149500,149100
4568,58207,1013,Chicopee,MA,Springfield,Hampden,4569,88500.0,88500.0,88400.0,...,167700,169300,171100,171900,172300,173300,174600,175900,176900,177400
2694,58209,1020,Chicopee,MA,Springfield,Hampden,2695,97200.0,97100.0,97000.0,...,178600,181000,184100,186100,186800,187400,188100,189000,189600,189800
14594,58212,1026,Cummington,MA,Springfield,Hampshire,14595,91200.0,91100.0,91000.0,...,193400,195900,200800,202600,201800,201200,204600,211200,217900,220800


In [21]:
df['RegionName'] = df.RegionName.astype(str)
df['RegionName'].dtypeb


dtype('O')

In [30]:

df_zips['Zip'] = df['RegionName']
df_zips.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,Zip
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,60657
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,310000,312500,314100,315000,316600,318100,319600,321100,321800,75070
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,320600,320200,320400,320800,321200,321200,323000,326900,329900,77494
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,60614
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119400,120000,120300,120300,120300,120300,120500,121000,121500,79936


In [39]:
# df_zips['Zip'] = ['0' + i for i in df_zips['RegionName'] if len(i) < 5]

zips = []

for i in df_zips['RegionName']:
    if len(i) < 5:
        i = '0' + i
        zips.append(i)
    else:
        zips.append(i)

df_zips['Zip'] = pd.Series(zips)
df_zips['Zip'].sort_values().head(10)


5850     01001
4199     01002
11213    01005
6850     01007
14547    01008
11702    01010
14475    01011
4568     01013
2694     01020
14594    01026
Name: Zip, dtype: object

In [41]:
df_zips.sort_values(by='Zip').head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,Zip
5850,58196,1001,Agawam,MA,Springfield,Hampden,5851,113100.0,112800.0,112600.0,...,215700,218200,220100,221100,221700,221700,221700,222700,223600,01001
4199,58197,1002,Amherst,MA,Springfield,Hampshire,4200,161000.0,160100.0,159300.0,...,334800,336700,338900,340400,342000,344500,347400,350600,353300,01002
11213,58200,1005,Barre,MA,Worcester,Worcester,11214,103100.0,103400.0,103600.0,...,206800,208800,210400,211300,213300,215600,217900,219500,220700,01005
6850,58201,1007,Belchertown,MA,Springfield,Hampshire,6851,133400.0,132700.0,132000.0,...,266300,267000,267500,268000,268100,268100,268800,270000,270600,01007
14547,58202,1008,Blandford,MA,Springfield,Hampden,14548,117500.0,117300.0,117100.0,...,202900,205900,208500,207500,205400,204500,206800,210900,214200,01008


# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

matplotlib.rc('font', **font)

df.plot(figsize=(12, 6));

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [56]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted   # removed remaining part of this code:  .groupby('time').aggregate({'value':'mean'})

In [57]:
df_melt = melt_data(df)
df_melt.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,1996-04-01,235700.0
2,91982,77494,Katy,TX,Houston,Harris,3,1996-04-01,210400.0
3,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
4,93144,79936,El Paso,TX,El Paso,El Paso,5,1996-04-01,77300.0


In [58]:
df_melt.groupby(['RegionName', 'time', 'State', 'City']).aggregate({'value':'mean'}).head(40)

value
RegionName time       State City            
1001       1996-04-01 MA    Agawam  113100.0
           1996-05-01 MA    Agawam  112800.0
           1996-06-01 MA    Agawam  112600.0
           1996-07-01 MA    Agawam  112300.0
           1996-08-01 MA    Agawam  112100.0
           1996-09-01 MA    Agawam  111900.0
           1996-10-01 MA    Agawam  111800.0
           1996-11-01 MA    Agawam  111700.0
           1996-12-01 MA    Agawam  111700.0
           1997-01-01 MA    Agawam  111900.0
           1997-02-01 MA    Agawam  112100.0
           1997-03-01 MA    Agawam  112300.0
           1997-04-01 MA    Agawam  112500.0
           1997-05-01 MA    Agawam  112800.0
           1997-06-01 MA    Agawam  113000.0
           1997-07-01 MA    Agawam  113300.0
           1997-08-01 MA    Agawam  113600.0
           1997-09-01 MA    Agawam  113900.0
           1997-10-01 MA    Agawam  114200.0
           1997-11-01 MA    Agawam  114500.0
           1997-12-01 MA    Agawam  114800.0
           1998-01-01 MA    Agawam  115200.0
           1998-02-01 MA    Agawam  115600.0
           1998-03-01 MA    Agawam  115900.0
           1998-04-01 MA    Agawam  116200.0
           1998-05-01 MA    Agawam  116400.0
           1998-06-01 MA    Agawam  116600.0
           1998-07-01 MA    Agawam  116700.0
           1998-08-01 MA    Agawam  116800.0
           1998-09-01 MA    Agawam  116900.0
           1998-10-01 MA    Agawam  117100.0
           1998-11-01 MA    Agawam  117300.0
           1998-12-01 MA    Agawam  117500.0
           1999-01-01 MA    Agawam  117800.0
           1999-02-01 MA    Agawam  118300.0
           1999-03-01 MA    Agawam  118700.0
           1999-04-01 MA    Agawam  119100.0
           1999-05-01 MA    Agawam  119500.0
           1999-06-01 MA    Agawam  120000.0
           1999-07-01 MA    Agawam  120400.0

In [62]:
df_melt.groupby(['RegionName', 'time', 'State', 'City']).sum().head(40)

RegionID  SizeRank     value
RegionName time       State City                                
1001       1996-04-01 MA    Agawam     58196      5851  113100.0
           1996-05-01 MA    Agawam     58196      5851  112800.0
           1996-06-01 MA    Agawam     58196      5851  112600.0
           1996-07-01 MA    Agawam     58196      5851  112300.0
           1996-08-01 MA    Agawam     58196      5851  112100.0
           1996-09-01 MA    Agawam     58196      5851  111900.0
           1996-10-01 MA    Agawam     58196      5851  111800.0
           1996-11-01 MA    Agawam     58196      5851  111700.0
           1996-12-01 MA    Agawam     58196      5851  111700.0
           1997-01-01 MA    Agawam     58196      5851  111900.0
           1997-02-01 MA    Agawam     58196      5851  112100.0
           1997-03-01 MA    Agawam     58196      5851  112300.0
           1997-04-01 MA    Agawam     58196      5851  112500.0
           1997-05-01 MA    Agawam     58196      5851  112800.0
           1997-06-01 MA    Agawam     58196      5851  113000.0
           1997-07-01 MA    Agawam     58196      5851  113300.0
           1997-08-01 MA    Agawam     58196      5851  113600.0
           1997-09-01 MA    Agawam     58196      5851  113900.0
           1997-10-01 MA    Agawam     58196      5851  114200.0
           1997-11-01 MA    Agawam     58196      5851  114500.0
           1997-12-01 MA    Agawam     58196      5851  114800.0
           1998-01-01 MA    Agawam     58196      5851  115200.0
           1998-02-01 MA    Agawam     58196      5851  115600.0
           1998-03-01 MA    Agawam     58196      5851  115900.0
           1998-04-01 MA    Agawam     58196      5851  116200.0
           1998-05-01 MA    Agawam     58196      5851  116400.0
           1998-06-01 MA    Agawam     58196      5851  116600.0
           1998-07-01 MA    Agawam     58196      5851  116700.0
           1998-08-01 MA    Agawam     58196      5851  116800.0
           1998-09-01 MA    Agawam     58196      5851  116900.0
           1998-10-01 MA    Agawam     58196      5851  117100.0
           1998-11-01 MA    Agawam     58196      5851  117300.0
           1998-12-01 MA    Agawam     58196      5851  117500.0
           1999-01-01 MA    Agawam     58196      5851  117800.0
           1999-02-01 MA    Agawam     58196      5851  118300.0
           1999-03-01 MA    Agawam     58196      5851  118700.0
           1999-04-01 MA    Agawam     58196      5851  119100.0
           1999-05-01 MA    Agawam     58196      5851  119500.0
           1999-06-01 MA    Agawam     58196      5851  120000.0
           1999-07-01 MA    Agawam     58196      5851  120400.0

In [30]:
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'}).head()

value
State City      time                     
AK    Anchorage 1996-04-01  139415.384615
                1996-05-01  139753.846154
                1996-06-01  140061.538462
                1996-07-01  140323.076923
                1996-08-01  140600.000000

In [31]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3744704 entries, 0 to 3901594
Data columns (total 9 columns):
RegionID      int64
RegionName    int64
City          object
State         object
Metro         object
CountyName    object
SizeRank      int64
time          datetime64[ns]
value         float64
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 285.7+ MB


In [32]:
df_melt.head()


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,time,value
0,84654,60657,Chicago,IL,Chicago,Cook,1,1996-04-01,334200.0
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,1996-04-01,235700.0
2,91982,77494,Katy,TX,Houston,Harris,3,1996-04-01,210400.0
3,84616,60614,Chicago,IL,Chicago,Cook,4,1996-04-01,498100.0
4,93144,79936,El Paso,TX,El Paso,El Paso,5,1996-04-01,77300.0


In [33]:
df_melt.set_index('time', inplace=True)

In [34]:
df_melt.head(10)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,value
time,,,,,,,,
1996-04-01,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0
1996-04-01,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0
1996-04-01,91982,77494,Katy,TX,Houston,Harris,3,210400.0
1996-04-01,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0
1996-04-01,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0
1996-04-01,91733,77084,Houston,TX,Houston,Harris,6,95000.0
1996-04-01,61807,10467,New York,NY,New York,Bronx,7,152900.0
1996-04-01,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0
1996-04-01,91940,77449,Katy,TX,Houston,Harris,9,95400.0


In [ ]:
df_melt.loc[df_melt['State'] == 'AK'].groupby(['time']).aggregate({'value':'mean'}).head()

In [45]:
df_melt['1997':].head(10)
df_melt.loc[df_melt['State'] == 'MA'].groupby(['RegionID', 'RegionName', 'time']).mean()

SizeRank     value
RegionID RegionName time                          
58196    1001       1996-04-01      5851  113100.0
                    1996-05-01      5851  112800.0
                    1996-06-01      5851  112600.0
                    1996-07-01      5851  112300.0
                    1996-08-01      5851  112100.0
                    1996-09-01      5851  111900.0
                    1996-10-01      5851  111800.0
                    1996-11-01      5851  111700.0
                    1996-12-01      5851  111700.0
                    1997-01-01      5851  111900.0
                    1997-02-01      5851  112100.0
                    1997-03-01      5851  112300.0
                    1997-04-01      5851  112500.0
                    1997-05-01      5851  112800.0
                    1997-06-01      5851  113000.0
                    1997-07-01      5851  113300.0
                    1997-08-01      5851  113600.0
                    1997-09-01      5851  113900.0
                    1997-10-01      5851  114200.0
                    1997-11-01      5851  114500.0
                    1997-12-01      5851  114800.0
                    1998-01-01      5851  115200.0
                    1998-02-01      5851  115600.0
                    1998-03-01      5851  115900.0
                    1998-04-01      5851  116200.0
                    1998-05-01      5851  116400.0
                    1998-06-01      5851  116600.0
                    1998-07-01      5851  116700.0
                    1998-08-01      5851  116800.0
                    1998-09-01      5851  116900.0
...                                  ...       ...
58904    2790       2015-11-01      5980  324200.0
                    2015-12-01      5980  326300.0
                    2016-01-01      5980  328000.0
                    2016-02-01      5980  329100.0
                    2016-03-01      5980  329900.0
                    2016-04-01      5980  331100.0
                    2016-05-01      5980  332400.0
                    2016-06-01      5980  334700.0
                    2016-07-01      5980  337500.0
                    2016-08-01      5980  340200.0
                    2016-09-01      5980  343700.0
                    2016-10-01      5980  347200.0
                    2016-11-01      5980  348700.0
                    2016-12-01      5980  348300.0
                    2017-01-01      5980  349100.0
                    2017-02-01      5980  351100.0
                    2017-03-01      5980  353200.0
                    2017-04-01      5980  354100.0
                    2017-05-01      5980  354600.0
                    2017-06-01      5980  354100.0
                    2017-07-01      5980  353900.0
                    2017-08-01      5980  354600.0
                    2017-09-01      5980  355100.0
                    2017-10-01      5980  357600.0
                    2017-11-01      5980  362700.0
                    2017-12-01      5980  368500.0
                    2018-01-01      5980  371600.0
                    2018-02-01      5980  373000.0
                    2018-03-01      5980  376900.0
                    2018-04-01      5980  382700.0

[110112 rows x 2 columns]

In [46]:
# df_melt_annual = df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
# df_melt_annual.loc[df_melt_annual['City'] =='Chicago'].aggregate({'value':'mean'})
                                # .groupby(['time']).aggregate({'value':'mean'}).head()
df_melt.groupby(['State', 'City', 'time']).aggregate({'value':'mean'})
df_melt.loc[df_melt['City']=='Chicago']

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,value
time,,,,,,,,
1996-04-01,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0
1996-04-01,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0
1996-04-01,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0
1996-04-01,84646,60647,Chicago,IL,Chicago,Cook,17,122700.0
1996-04-01,84620,60618,Chicago,IL,Chicago,Cook,20,142600.0
1996-04-01,84615,60613,Chicago,IL,Chicago,Cook,41,297900.0
1996-04-01,84630,60629,Chicago,IL,Chicago,Cook,49,93400.0
1996-04-01,84627,60625,Chicago,IL,Chicago,Cook,51,148900.0
1996-04-01,84621,60619,Chicago,IL,Chicago,Cook,75,76400.0


# Step 5: ARIMA Modeling

# Step 6: Interpreting Results